# VacationPy
----


In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import warnings
warnings.filterwarnings("ignore")

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities = pd.read_csv('../WeatherPy/cities.csv') 
cities.head()

,City,Lat,Lngs,Temp,Humidity,Cloudiness,Wind Speed
0,castro,-24.79,-50.01,63.37,96,100,8.21
1,bluff,-46.60,168.33,55.99,68,99,5.99
2,busselton,-33.65,115.33,67.35,43,0,13.00
3,hermanus,-34.42,19.23,61.00,85,100,14.03
4,los llanos de aridane,28.66,-17.92,64.40,63,40,5.82


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [4]:
locations =cities[["Lat", "Lngs"]].astype(float)
humidity =cities["Humidity"].astype(float)

In [5]:
# Create a poverty Heatmap layer
fig = gmaps.figure()

heatmap = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 5)

fig.add_layer(heatmap)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
desireWeather = (cities["Temp"] > 70) & (cities["Temp"] < 90) & \
    (cities["Humidity"] < 50) & (cities["Wind Speed"] < 20)

desireWeather =cities[desireWeather]
desireWeather.head()

,City,Lat,Lngs,Temp,Humidity,Cloudiness,Wind Speed
16,new norfolk,-42.78,147.06,71.01,32,69,3.00
39,geraldton,-28.77,114.60,73.40,38,0,19.46
48,hobart,-42.88,147.33,71.83,32,0,6.93
103,otjimbingwe,-22.35,16.13,71.28,33,78,0.96
109,gao,16.64,1.64,70.86,20,0,8.32


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
coorsDf = desireWeather[["Lat", "Lngs"]].astype(float)
coors = coorsDf.values.tolist()

targetSearch = "Hotel"
targetRadius = 5000

hotelName = []

for coor in coors:
    
    baseUrl = (f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?\
location={[coor][0][0]},{[coor][0][1]}&radius={targetRadius}&keyword={targetSearch}&key={g_key}")
    response = requests.get(baseUrl)
    hotelData = response.json()
    try:
        hotelName.append((hotelData["results"][0]["name"]))
    except:
        hotelName.append("NA")
    

In [8]:
desireWeather["Hotel Name"] = hotelName
desireWeather.head()

,City,Lat,Lngs,Temp,Humidity,Cloudiness,Wind Speed,Hotel Name
16,new norfolk,-42.78,147.06,71.01,32,69,3.00,Tynwald Willow Bend Estate
39,geraldton,-28.77,114.60,73.40,38,0,19.46,Ocean Centre Hotel
48,hobart,-42.88,147.33,71.83,32,0,6.93,Wrest Point
103,otjimbingwe,-22.35,16.13,71.28,33,78,0.96,NA
109,gao,16.64,1.64,70.86,20,0,8.32,NA


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in desireWeather.iterrows()]
locations = desireWeather[["Lat", "Lngs"]]

In [10]:
# Add marker layer ontop of heat map
markerLayer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig = gmaps.figure()
fig.add_layer(markerLayer)
fig.add_layer(heatmap)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))